# Prosopo Training Notebook

Train a face embedding model from scratch using ArcFace loss.

**Target:** 99%+ accuracy on LFW benchmark

---

## Pipeline Overview
1. Mount Drive (checkpoint survival)
2. Download CASIA-WebFace from Kaggle (.rec format)
3. Unpack RecordIO → raw images
4. Align faces with MTCNN → 112×112
5. Train ResNet-50 + ArcFace
6. Evaluate on LFW

## 1. Setup & Mount Drive

⚠️ **CRITICAL:** Mount Drive FIRST to ensure checkpoints survive session disconnects.

In [1]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

# Create directories
import os
os.makedirs('/content/drive/MyDrive/prosopo/checkpoints', exist_ok=True)
os.makedirs('/content/data', exist_ok=True)
print('✅ Drive mounted')

Mounted at /content/drive
✅ Drive mounted


In [2]:
# Install dependencies
# CRITICAL: Force numpy<2.0 because mxnet and facenet-pytorch are broken on numpy 2.x
!pip install -q "numpy<2.0.0"
!pip install -q torch torchvision
!pip install -q albumentations facenet-pytorch scikit-image
!pip install -q tqdm scikit-learn opencv-python
!pip install -q mxnet  # For RecordIO unpacking
!pip install -q kaggle

print('✅ Dependencies installed')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 83.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9

In [3]:
# Clone Prosopo repo and verify structure
!git clone https://github.com/InanXR/Prosopo.git /content/prosopo

# Verify the package exists
import os
expected_files = [
    '/content/prosopo/prosopo/__init__.py',
    '/content/prosopo/prosopo/models/arcface.py',
    '/content/prosopo/prosopo/training/trainer.py',
    '/content/prosopo/prosopo/data/dataset.py',
    '/content/prosopo/scripts/preprocess.py',
]

all_ok = True
for f in expected_files:
    if os.path.exists(f):
        print(f'✅ {f}')
    else:
        print(f'❌ MISSING: {f}')
        all_ok = False

if all_ok:
    print('\n✅ Prosopo repo verified!')
else:
    print('\n❌ Some files missing - check GitHub repo!')

Cloning into '/content/prosopo'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 47 (delta 9), reused 45 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 28.99 KiB | 5.80 MiB/s, done.
Resolving deltas: 100% (9/9), done.
✅ /content/prosopo/prosopo/__init__.py
✅ /content/prosopo/prosopo/models/arcface.py
✅ /content/prosopo/prosopo/training/trainer.py
✅ /content/prosopo/prosopo/data/dataset.py
✅ /content/prosopo/scripts/preprocess.py

✅ Prosopo repo verified!


## 2. Setup Kaggle API

Using Global Access Token (KGAT) method.

In [4]:
# Configure Kaggle credentials
import os
import json

# 1. Define credentials
kaggle_username = 'inanxr'
kaggle_token = 'KGAT_1200fe88c38a44a77c8879998f9413ac'

# 2. Manually create kaggle.json to avoid client errors
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)

with open(os.path.join(kaggle_dir, 'kaggle.json'), 'w') as f:
    # Note: KGAT acts as the 'key'
    json.dump({'username': kaggle_username, 'key': kaggle_token}, f)

# 3. Set permissions
!chmod 600 ~/.kaggle/kaggle.json

print('✅ Kaggle credentials configured')

# 4. Verify it works
!kaggle datasets list --sort-by votes 2>&1 | head -n 3

✅ Kaggle credentials configured
ref                                                           title                                                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
jessicali9530/animal-crossing-new-horizons-nookplaza-dataset  Animal Crossing New Horizons Catalog                     590770  2021-06-08 15:05:09.513000          83596      56002  0.88235295       


## 3. Download CASIA-WebFace from Kaggle

Dataset: `debarghamitraroy/casia-webface` (~2.73 GB)

In [5]:
# Download CASIA-WebFace dataset
!kaggle datasets download -d debarghamitraroy/casia-webface -p /content/data/

print('\n✅ Download complete. Checking contents...')
!ls -lh /content/data/

Dataset URL: https://www.kaggle.com/datasets/debarghamitraroy/casia-webface
License(s): DbCL-1.0
 99% 2.67G/2.70G [00:17<00:00, 256MB/s]
100% 2.70G/2.70G [00:17<00:00, 163MB/s]

✅ Download complete. Checking contents...
total 2.7G
-rw-r--r-- 1 root root 2.7G Dec 22 13:33 casia-webface.zip


In [6]:
# Unzip and verify structure
!unzip -q /content/data/casia-webface.zip -d /content/data/raw_rec

print('\nContents after unzip:')
!find /content/data/raw_rec -name "*.rec" -o -name "*.idx" | head -20

# Find the .rec file path
import glob
rec_files = glob.glob('/content/data/raw_rec/**/*.rec', recursive=True)
print(f'\nFound .rec files: {rec_files}')


Contents after unzip:
/content/data/raw_rec/casia-webface/train.idx
/content/data/raw_rec/casia-webface/train.rec

Found .rec files: ['/content/data/raw_rec/casia-webface/train.rec']


## 4. Unpack RecordIO to Raw Images

The dataset comes in MXNet RecordIO format. We unpack it to folders.

⏱️ **Time:** ~30 minutes for 490K images

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# --- 🛠️ HOTFIX: Patch NumPy for MXNet Compatibility 🛠️ ---
# MXNet is an older library and looks for 'np.bool' and 'np.object', which
# were removed in modern NumPy. We manually restore them here to prevent crashes.
if not hasattr(np, 'bool'):
    np.bool = bool
if not hasattr(np, 'object'):
    np.object = object
if not hasattr(np, 'float'):
    np.float = float
# -----------------------------------------------------------

import mxnet as mx
from mxnet import recordio

def unpack_rec_file(rec_path, output_dir):
    """
    Unpack MXNet RecordIO file to image folders using OpenCV for decoding.
    """
    print(f"Unpacking {rec_path}...")

    idx_path = rec_path.replace('.rec', '.idx')
    if not os.path.exists(rec_path):
        raise FileNotFoundError(f"{rec_path} not found!")
    if not os.path.exists(idx_path):
        raise FileNotFoundError(f"{idx_path} not found! (Required alongside .rec)")

    # Open RecordIO
    imgrec = recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')

    # Read header to get total count
    s = imgrec.read_idx(0)
    header, _ = recordio.unpack(s)

    if isinstance(header.label, float):
        num_images = int(header.label)
    else:
        num_images = int(header.label[0])

    print(f"Total images: {num_images:,}")
    os.makedirs(output_dir, exist_ok=True)

    success_count = 0
    error_count = 0

    # Iterate and unpack
    for idx in tqdm(range(1, num_images + 1), desc="Unpacking", mininterval=1.0):
        try:
            s = imgrec.read_idx(idx)
            header, img_data = recordio.unpack(s)

            # --- OPTIMIZATION: Use OpenCV directly instead of MXNet ---
            # 1. Convert bytes to numpy array
            nparr = np.frombuffer(img_data, dtype=np.uint8)
            # 2. Decode directly to BGR (ready for imwrite)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            if img is None:
                raise ValueError("Image decode failed")

            # Handle label format
            if isinstance(header.label, float):
                label = int(header.label)
            else:
                label = int(header.label[0])

            # Create folder structure
            folder_name = f"{label:07d}"
            save_dir = os.path.join(output_dir, folder_name)
            os.makedirs(save_dir, exist_ok=True)

            # Save image
            filename = f"{idx}.jpg"
            cv2.imwrite(os.path.join(save_dir, filename), img)
            success_count += 1

        except Exception:
            error_count += 1
            continue

    print(f"\n✅ Unpacked {success_count:,} images ({error_count} errors)")
    print(f"   Output: {output_dir}")
    print(f"   Identities: {len(os.listdir(output_dir))}")

/tmp/ipython-input-3375138610.py:9: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):
/tmp/ipython-input-3375138610.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'):


In [2]:
# Find and unpack the .rec file
import glob

rec_files = glob.glob('/content/data/raw_rec/**/*.rec', recursive=True)

if not rec_files:
    print("❌ No .rec file found! Checking directory structure...")
    !find /content/data -type f | head -30
else:
    rec_path = rec_files[0]
    print(f"Using: {rec_path}")
    unpack_rec_file(rec_path, '/content/data/raw_casia')

Using: /content/data/raw_rec/casia-webface/train.rec
Unpacking /content/data/raw_rec/casia-webface/train.rec...
Total images: 490,624


Unpacking: 100%|██████████| 490624/490624 [03:03<00:00, 2667.09it/s]


✅ Unpacked 490,623 images (1 errors)
   Output: /content/data/raw_casia
   Identities: 10572


## 5. Align Faces with MTCNN

Detect faces and warp to canonical 112×112 pose.

⏱️ **Time:** ~2-4 hours for 490K images

In [ ]:
import torch
from facenet_pytorch import MTCNN
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageOps
import os
import glob
from tqdm import tqdm
import json
import numpy as np

# --- CONFIGURATION ---
INPUT_ROOT = '/content/data/raw_casia'
OUTPUT_ROOT = '/content/data/aligned_casia'
BATCH_SIZE = 64     # Reduced slightly for safety with resizing
NUM_WORKERS = 2
TARGET_SIZE = 640   # Standardize inputs to this size for batching

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Acceleration: {device}")

# Initialize MTCNN
mtcnn = MTCNN(
    image_size=112,
    margin=0,
    min_face_size=20,
    thresholds=[0.6, 0.7, 0.7],
    factor=0.709,
    post_process=True,
    device=device
)

# Helper: Resize image to square with padding (preserves aspect ratio)
def resize_pad(img, target_size):
    old_size = img.size  # (width, height)
    ratio = float(target_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    img = img.resize(new_size, Image.BILINEAR)

    # Create new square image and paste resized img in center
    new_im = Image.new("RGB", (target_size, target_size))
    new_im.paste(img, ((target_size-new_size[0])//2,
                       (target_size-new_size[1])//2))
    return new_im

class FaceDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        try:
            img = Image.open(path).convert('RGB')
            # Critical Fix: Standardize size so they can be batched!
            img = resize_pad(img, TARGET_SIZE)
        except Exception:
            return None, None
        return img, path

def collate_fn(batch):
    batch = [b for b in batch if b[0] is not None]
    if len(batch) == 0:
        return [], []
    return zip(*batch)

# Collect Files
files = glob.glob(f"{INPUT_ROOT}/**/*.jpg", recursive=True)
print(f"📂 Found {len(files):,} images. Processing...")

# Run Batch Alignment
dataset = FaceDataset(files)
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

aligned_count = 0
errors = 0

for imgs, paths in tqdm(loader, desc="Aligning", unit="batch"):
    if not imgs: continue

    save_paths = []
    for p in paths:
        rel_path = os.path.relpath(p, INPUT_ROOT)
        out_path = os.path.join(OUTPUT_ROOT, rel_path)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        save_paths.append(out_path)

    try:
        # Convert tuple to list for MTCNN
        mtcnn(list(imgs), save_path=save_paths)
        aligned_count += len(imgs)
    except Exception as e:
        errors += 1
        continue

print(f"\n✨ Complete! Processed {aligned_count:,} images.")

# Generate class_indices.json
classes = [d for d in os.listdir(OUTPUT_ROOT) if os.path.isdir(os.path.join(OUTPUT_ROOT, d))]
classes.sort()
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

json_path = os.path.join(OUTPUT_ROOT, 'class_indices.json')
with open(json_path, 'w') as f:
    json.dump(class_to_idx, f, indent=4)
print(f"✅ Saved class indices. Total Classes: {len(classes)}")

🚀 Acceleration: cuda:0
📂 Found 490,623 images. Processing...


Aligning:   3%|▎         | 264/7666 [05:02<2:28:52,  1.21s/batch]

In [ ]:
# Verify alignment results
import os

aligned_dir = '/content/data/aligned_casia'

if os.path.exists(aligned_dir):
    num_identities = len([d for d in os.listdir(aligned_dir)
                          if os.path.isdir(os.path.join(aligned_dir, d))])

    total_images = sum(len(files) for _, _, files in os.walk(aligned_dir))

    print(f"✅ Alignment complete!")
    print(f"   Identities: {num_identities:,}")
    print(f"   Total aligned images: {total_images:,}")

    # Check for class_indices.json
    if os.path.exists(f"{aligned_dir}/class_indices.json"):
        print(f"   ✅ class_indices.json exists")
    else:
        print(f"   ⚠️ class_indices.json not found")
else:
    print("❌ Aligned directory not found!")

In [ ]:
import shutil
import os

# Define paths
source_dir = '/content/data/aligned_casia'
backup_path = '/content/drive/MyDrive/prosopo/aligned_casia.zip'

print(f"📦 Zipping dataset... this may take a few minutes...")
shutil.make_archive(backup_path.replace('.zip', ''), 'zip', source_dir)

print(f"✅ Backup saved to: {backup_path}")
print("   (If the session crashes, you can just unzip this file instead of realigning!)")

## 6. Download LFW for Evaluation

In [ ]:
# Download LFW
!wget -q http://vis-www.cs.umass.edu/lfw/lfw.tgz -O /content/data/lfw.tgz
!tar -xzf /content/data/lfw.tgz -C /content/data/
!wget -q http://vis-www.cs.umass.edu/lfw/pairs.txt -O /content/data/pairs.txt

print('✅ LFW downloaded')
!ls /content/data/lfw | head -5

## 7. Configure Training

In [ ]:
import sys
sys.path.insert(0, '/content/prosopo')

# Test import
try:
    from prosopo.training import TrainingConfig, Trainer
    from prosopo.models import Prosopo
    print('✅ Prosopo imports successful')
except ImportError as e:
    print(f'❌ Import failed: {e}')
    print('\nCheck that all files were pushed to GitHub!')

In [ ]:
from prosopo.training import TrainingConfig

config = TrainingConfig(
    # Data paths
    data_root='/content/data/aligned_casia',
    class_indices_path='/content/data/aligned_casia/class_indices.json',
    lfw_root='/content/data/lfw',
    lfw_pairs_path='/content/data/pairs.txt',

    # Model
    backbone='resnet50',
    embedding_dim=512,
    pretrained=True,

    # ArcFace
    arcface_scale=64.0,
    arcface_margin=0.5,

    # Training
    batch_size=128,
    accumulation_steps=2,
    epochs=25,
    lr=0.1,
    num_workers=2,

    # Checkpointing
    checkpoint_dir='/content/drive/MyDrive/prosopo/checkpoints',
    save_every=1,
    val_epochs=[10, 15, 20, 25],

    # Resume (set path if session crashed)
    resume_from=None,  # e.g., '/content/drive/MyDrive/prosopo/checkpoints/epoch_10.pth'
)

print('✅ Config ready')
print(f'   Batch size: {config.batch_size} x {config.accumulation_steps} = {config.batch_size * config.accumulation_steps} effective')

## 8. Train Model

⏱️ **Expected time:** ~8-12 hours on T4 GPU

If session disconnects:
1. Re-run cells 1-6 (they're fast - data is cached)
2. Set `resume_from` to last checkpoint path
3. Re-run training

In [ ]:
import torch
print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
from prosopo.training import Trainer

trainer = Trainer(config)
trainer.train()

## 9. Final Evaluation

In [ ]:
from prosopo.evaluation import evaluate_lfw

accuracy, threshold = evaluate_lfw(
    trainer.model,
    config.lfw_root,
    config.lfw_pairs_path,
)

print(f'\n🎯 LFW Accuracy: {accuracy:.2%}')
print(f'   Optimal threshold: {threshold:.3f}')

## 10. Export Model

In [ ]:
import torch

final_path = '/content/drive/MyDrive/prosopo/prosopo_final.pth'
torch.save(trainer.model.state_dict(), final_path)
print(f'✅ Model saved to: {final_path}')

In [ ]:
from google.colab import files
files.download(final_path)